In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from pathlib import Path
from pprint import pprint

import lerobot
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset
from lerobot.common.datasets.factory import make_dataset


from hydra import compose, initialize
from omegaconf import OmegaConf

# context initialization
with initialize(version_base=None, config_path="../configs", job_name="test_app"):
    cfg = compose(config_name="default")
    print(OmegaConf.to_yaml(cfg))

resume: false
device: cuda
use_amp: false
seed: 100000
dataset_repo_id: lerobot/pusht
training:
  offline_steps: 200000
  online_steps: 0
  online_steps_between_rollouts: 1
  online_sampling_ratio: 0.5
  online_env_seed: ???
  eval_freq: 5000
  save_freq: 5000
  log_freq: 250
  save_checkpoint: true
  num_workers: 4
  batch_size: 64
  grad_clip_norm: 10
  lr: 0.0001
  lr_scheduler: cosine
  lr_warmup_steps: 500
  adam_betas:
  - 0.95
  - 0.999
  adam_eps: 1.0e-08
  adam_weight_decay: 1.0e-06
  delta_timestamps:
    observation.image: '[i / ${fps} for i in range(1 - ${policy.n_obs_steps}, 1)]'
    observation.state: '[i / ${fps} for i in range(1 - ${policy.n_obs_steps}, 1)]'
    action: '[i / ${fps} for i in range(1 - ${policy.n_obs_steps}, 1 - ${policy.n_obs_steps}
      + ${policy.horizon})]'
  drop_n_last_frames: 7
eval:
  n_episodes: 50
  batch_size: 50
  use_async_envs: false
wandb:
  enable: false
  disable_artifact: false
  project: lerobot
  notes: ''
tensorboard:
  enable: true

In [13]:
# get the path to the dataset
import pandas as pd
import numpy as np
from pathlib import Path
env_name = 'robosuite' # 'pinpad'

# base_path = Path(f"~/workspace/lerobot/local/{env_name}/original").expanduser()
base_path = Path(f"~/workspace/fastrl/logs/HD_robosuite_Square_D0_0/augmented").expanduser()
out_dir = Path(f"~/workspace/lerobot/local/{env_name}").expanduser()

# list all the files in the dataset
files = list(base_path.glob("*"))
# for f in files:
#     print(f)

# print the keys
data = np.load(files[0])
# convert to a dictionary NOTE: this is necessary to make the arrays writeable for some reason
data = dict(data)
for k,v in data.items():
    print(k, v.shape)

# print("Setting last is_terminal to true")
# data["is_terminal"][-1] = True; data['is_last'][-1] = True

action (162, 7)
reward (162,)
is_first (162,)
is_last (162,)
is_terminal (162,)
image (162, 128, 128, 3)
alt_image (162, 128, 128, 3)
vector_state (162, 21)
discount (162,)
logprob (162,)


In [21]:
import tqdm
import torch
import einops
import shutil
from PIL import Image as PILImage
import cv2

from lerobot.common.datasets.push_dataset_to_hub.utils import concatenate_episodes, save_images_concurrently
from lerobot.common.datasets.compute_stats import compute_stats
from lerobot.scripts.push_dataset_to_hub import save_meta_data
from lerobot.common.datasets.video_utils import VideoFrame, encode_video_frames
from lerobot.common.datasets.utils import hf_transform_to_torch
from datasets import Dataset, Features, Image, Sequence, Value

def to_hf_dataset(data_dict, video):
    features = {}

    if video:
        features["observation.image"] = VideoFrame()
    else:
        features["observation.image"] = Image()

    features["observation.state"] = Sequence(
        length=data_dict["observation.state"].shape[1], feature=Value(dtype="float32", id=None)
    )
    features["action"] = Sequence(
        length=data_dict["action"].shape[1], feature=Value(dtype="float32", id=None)
    )
    features["episode_index"] = Value(dtype="int64", id=None)
    features["frame_index"] = Value(dtype="int64", id=None)
    features["timestamp"] = Value(dtype="float32", id=None)
    features["next.reward"] = Value(dtype="float32", id=None)
    features["next.done"] = Value(dtype="bool", id=None)
    features["index"] = Value(dtype="int64", id=None)
    # TODO(rcadene): add success
    # features["next.success"] = Value(dtype='bool', id=None)

    hf_dataset = Dataset.from_dict(data_dict, features=Features(features))
    hf_dataset.set_transform(hf_transform_to_torch)
    return hf_dataset

video = False; fps = 20; video_path = None
debug = False


ep_dicts = []
episode_data_index = {"from": [], "to": []}

id_from = 0
id_to = 0
ep_idx = 0

data_dicts = []
for data_fn in files:
    print(f"Processing {data_fn}")
    data = np.load(data_fn)
    data = dict(data); 
    data["is_terminal"][-1] = True
    data_dicts.append(data)
big_data_dict = {}
for k in data_dicts[0].keys():
    big_data_dict[k] = np.concatenate([d[k] for d in data_dicts], axis=0)
    print(k, big_data_dict[k].shape)

Processing /home/j/workspace/fastrl/logs/HD_robosuite_Square_D0_0/augmented/demo_255.npz
Processing /home/j/workspace/fastrl/logs/HD_robosuite_Square_D0_0/augmented/demo_215.npz
Processing /home/j/workspace/fastrl/logs/HD_robosuite_Square_D0_0/augmented/demo_278.npz
Processing /home/j/workspace/fastrl/logs/HD_robosuite_Square_D0_0/augmented/demo_25.npz
Processing /home/j/workspace/fastrl/logs/HD_robosuite_Square_D0_0/augmented/demo_211.npz
Processing /home/j/workspace/fastrl/logs/HD_robosuite_Square_D0_0/augmented/demo_261.npz
Processing /home/j/workspace/fastrl/logs/HD_robosuite_Square_D0_0/augmented/demo_247.npz
Processing /home/j/workspace/fastrl/logs/HD_robosuite_Square_D0_0/augmented/demo_196.npz
Processing /home/j/workspace/fastrl/logs/HD_robosuite_Square_D0_0/augmented/demo_234.npz
Processing /home/j/workspace/fastrl/logs/HD_robosuite_Square_D0_0/augmented/demo_244.npz
Processing /home/j/workspace/fastrl/logs/HD_robosuite_Square_D0_0/augmented/demo_170.npz
Processing /home/j/wor

In [22]:

data = big_data_dict
total_frames = data["action"].shape[0]
# for i in tqdm.tqdm(range(total_frames)):
for i in range(total_frames):
    id_to += 1

    if not data["is_terminal"][i]:
        continue

    print("found terminal step")

    num_frames = id_to - id_from

    image = torch.tensor(data["image"][id_from:id_to])
    # image = einops.rearrange(image, "b h w c -> b h w c")
    # image = einops.rearrange(image, "b c h w -> b h w c")
    state = torch.tensor(data["vector_state"][id_from:id_to]) if ("vector_state" in data) else torch.zeros(num_frames, 1)
    action = torch.tensor(data["action"][id_from:id_to])
    # TODO(rcadene): we have a missing last frame which is the observation when the env is done
    # it is critical to have this frame for tdmpc to predict a "done observation/state"
    # next_image = torch.tensor(data["next_observations"]["rgb"][id_from:id_to])
    # next_state = torch.tensor(data["next_observations"]["state"][id_from:id_to])
    next_reward = torch.tensor(data["reward"][id_from:id_to])
    next_done = torch.tensor(data["is_terminal"][id_from:id_to])

    ep_dict = {}

    imgs_array = [x.numpy() for x in image]
    img_key = "observation.image"
    if video:
        # save png images in temporary directory
        tmp_imgs_dir = out_dir / "tmp_images"
        tmp_imgs_dir.mkdir(parents=True, exist_ok=True)

        for i in range(len(imgs_array)):
            img = PILImage.fromarray(imgs_array[i])
            img.save(str(tmp_imgs_dir / f"frame_{i:06d}.png"), quality=100)

        # encode images to a mp4 video
        fname = f"{img_key}_episode_{ep_idx:06d}.mp4"
        video_path = out_dir / "videos" / fname
        encode_video_frames(tmp_imgs_dir, video_path, fps)

        # clean temporary images directory
        shutil.rmtree(tmp_imgs_dir)

        # store the reference to the video frame
        ep_dict[img_key] = [{"path": f"videos/{fname}", "timestamp": i / fps} for i in range(num_frames)]
    else:
        # ep_dict[img_key] = [PILImage.fromarray(x) for x in imgs_array]
        ep_dict[img_key] = imgs_array

    ep_dict["observation.state"] = state
    ep_dict["action"] = action
    ep_dict["episode_index"] = torch.tensor([ep_idx] * num_frames, dtype=torch.int64)
    ep_dict["frame_index"] = torch.arange(0, num_frames, 1)
    ep_dict["timestamp"] = torch.arange(0, num_frames, 1) / fps
    # ep_dict["next.observation.image"] = next_image
    # ep_dict["next.observation.state"] = next_state
    ep_dict["next.reward"] = next_reward
    ep_dict["next.done"] = next_done
    ep_dicts.append(ep_dict)

    episode_data_index["from"].append(id_from)
    episode_data_index["to"].append(id_from + num_frames)

    id_from = id_to
    ep_idx += 1

    # process first episode only
    if debug:
        break
if len(ep_dicts) == 0:
    print("No terminal step found in the dataset")
else:
    data_dict = concatenate_episodes(ep_dicts)
    data_dict, episode_data_index

    hf_dataset = to_hf_dataset(data_dict, video)

    info = {"fps": fps, "video": video}

    if video_path: 
        print(f"video path: {video_path}")
    lerobot_dataset = LeRobotDataset.from_preloaded(
    repo_id="pinpad",
    version=0.0,
    hf_dataset=hf_dataset,
    episode_data_index=episode_data_index,
    info=info,
    videos_dir=video_path,
    )


    hf_dataset = hf_dataset.with_format(None)  # to remove transforms that cant be saved
    hf_dataset.save_to_disk(str(out_dir / "train"))
print(lerobot_dataset)
stats = compute_stats(lerobot_dataset, batch_size=16, num_workers=1)
save_meta_data(info, stats, episode_data_index, out_dir / "meta_data")


found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step
found terminal step


Saving the dataset (1/1 shards): 100%|██████████| 30376/30376 [00:01<00:00, 24044.91 examples/s]

LeRobotDataset(
  Repository ID: 'pinpad',
  Version: '0.0',
  Split: 'train',
  Number of Samples: 30376,
  Number of Episodes: 200,
  Type: image (.png),
  Recorded Frames per Second: 20,
  Camera Keys: ['observation.image'],
  Video Frame Keys: N/A,
  Transformations: None,
)



Compute std: 100%|█████████▉| 1898/1899 [00:33<00:00, 57.07it/s]


In [9]:

from datasets import load_dataset, load_from_disk
loaded_dataset = load_from_disk(str(out_dir / "train"))

dataloader = torch.utils.data.DataLoader(
    loaded_dataset,
    num_workers=1,
    batch_size=2,
    shuffle=False,
)

batch = next(iter(dataloader))

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/j/workspace/lerobot/venv/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 154, in collate
    clone.update({key: collate([d[key] for d in batch], collate_fn_map=collate_fn_map) for key in elem})
  File "/home/j/workspace/lerobot/venv/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 154, in <dictcomp>
    clone.update({key: collate([d[key] for d in batch], collate_fn_map=collate_fn_map) for key in elem})
  File "/home/j/workspace/lerobot/venv/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 191, in collate
    raise TypeError(default_collate_err_msg_format.format(elem_type))
TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.PngImagePlugin.PngImageFile'>

During handling of the above exception, another exception occurred:

Traceback (most recent